## TLDR:
#### SNP List:
###### * eQTLs loaded from GM12878 significant variants GTEX file
###### * Top genes (from 01 - Top genes notebook) merged with eQTLs to obtain top variant-gene pairs.
#### Filtering
###### * Same pairs are read for all GTEX tissues (missing non-significant pairs for such tissues)
###### * Each tissue data is Z normalized (scipy) with sign modified to represent direction of eQTL.
###### * Z normalized eQTLs are saved and all tissues merged together
#### Gene intersection
###### * Pairs are intersected with SNP database to obtain rs id and other data
#### LD
###### * For each gene in variant-gene pair, Postgap is used to obtain LD scores of all variants paired to the gene
###### * R, R2 and Z values for each gene are saved in individual files for each tissue.

In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import time
import postgap
import postgap.Globals
import postgap.Integration
from postgap.Utils import *
import requests
import os
import scipy.stats as stats

# SNP list

In [2]:
eQTL=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/GTEX/GTEx_Analysis_v8_eQTL/Cells_EBV-transformed_lymphocytes.v8.signif_variant_gene_pairs.txt.gz',compression='gzip',sep='\t')

In [3]:
eQTL['gene_id']=eQTL['gene_id'].str.split('.',expand=True)[0]

In [4]:
eQTL

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta
0,chr1_739465_TTTTG_T_b38,ENSG00000237094,237848,5,6,0.020408,1.425050e-05,1.603600,0.354555,0.000043,4.470720e-06,6.332260e-03
1,chr1_763097_C_T_b38,ENSG00000237094,261480,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
2,chr1_763107_A_G_b38,ENSG00000237094,261490,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
3,chr1_774708_C_A_b38,ENSG00000237094,273091,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
4,chr1_775065_A_G_b38,ENSG00000237094,273448,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
5,chr1_777712_G_A_b38,ENSG00000237094,276095,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
6,chr1_778679_C_CCGCCCTTGTGACGTCACGGAAGGCG_b38,ENSG00000237094,277062,5,5,0.018657,1.602060e-05,2.018330,0.449131,0.000043,4.470720e-06,6.332260e-03
7,chr1_782094_C_T_b38,ENSG00000237094,280477,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
8,chr1_795668_C_T_b38,ENSG00000237094,294051,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03
9,chr1_796444_A_G_b38,ENSG00000237094,294827,5,5,0.017007,2.038260e-05,1.992520,0.449392,0.000043,4.470720e-06,6.332260e-03


In [5]:
gene_list=pd.DataFrame(eQTL['gene_id'].str.split('.',expand=True)[0].unique(),columns=['Name'])

In [6]:
top_genes=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/top_genes_GM12878.csv.gz',compression='gzip',names=['gene_id','Description','GM12878-Median-Expression'])

In [7]:
top_genes_eQTLs=top_genes.merge(eQTL)

In [8]:
genes_variants=top_genes_eQTLs[['gene_id','variant_id']]

In [9]:
genes_variants.to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/gene_variant_selection.csv',index=False)

# Filtering same genes and eQTLs for each tissue

In [2]:
extension='.v8.signif_variant_gene_pairs.txt.gz'
in_folder='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/GTEX/GTEx_Analysis_v8_eQTL/'
out_folder_z='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/eQTL_tissues/eQTLs_z/'
out_folder='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/eQTL_tissues/'

In [7]:
tissues=[x.replace(extension,'') for x in os.listdir(in_folder) if x.endswith(extension)]

In [17]:
for i in tissues:
    file=in_folder+i+extension
    folder=out_folder_z+i+'/'
    if not os.path.exists(folder):os.makedirs(folder)
    tissue_eQTL=pd.read_csv(file,sep='\t')
    tissue_eQTL['gene_id']=tissue_eQTL['gene_id'].str.rsplit('.',expand=True)[0]
    tissue_eQTL=tissue_eQTL.merge(genes_variants)[['variant_id','gene_id','pval_nominal','slope']]
    tissue_eQTL['slope'][tissue_eQTL['slope'] > 0] = 1
    tissue_eQTL['slope'][tissue_eQTL['slope'] < 0] = -1
    tissue_eQTL[i+'_z']=stats.zscore(tissue_eQTL['pval_nominal'])*tissue_eQTL['slope']
    #tissue_eQTL=tissue_eQTL.rename(columns={'pval_nominal':i+'_pval_nominal'})
    tissue_eQTL=tissue_eQTL.drop(axis=1,labels=['slope','pval_nominal'])
    tissue_eQTL.to_csv(folder+i+'_ALL_z.csv.gz',index=False,compression='gzip')

/homes/jhidalgo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
eQTLs=pd.DataFrame()
for i in tissues:
    file=out_folder+i+'/'+i+'_ALL_z.csv.gz'
    data=pd.read_csv(file,compression='gzip')
    if eQTLs.empty:eQTLs=data
    else: eQTLs=eQTLs.merge(data,how='outer')

In [37]:
eQTLs

NameError: name 'eQTLs' is not defined

In [21]:
eQTLs.corr()

,Adipose_Subcutaneous_z,Adipose_Visceral_Omentum_z,Adrenal_Gland_z,Artery_Aorta_z,Artery_Coronary_z,Artery_Tibial_z,Brain_Amygdala_z,Brain_Anterior_cingulate_cortex_BA24_z,Brain_Caudate_basal_ganglia_z,Brain_Cerebellar_Hemisphere_z,...,Skin_Not_Sun_Exposed_Suprapubic_z,Skin_Sun_Exposed_Lower_leg_z,Small_Intestine_Terminal_Ileum_z,Spleen_z,Stomach_z,Testis_z,Thyroid_z,Uterus_z,Vagina_z,Whole_Blood_z
Adipose_Subcutaneous_z,1.000000,0.142561,0.077067,0.216581,0.090789,0.138014,0.027970,0.127328,0.244253,0.062549,...,0.049750,0.091651,0.019873,0.158412,0.157318,0.060519,0.114693,0.009298,0.031946,0.112354
Adipose_Visceral_Omentum_z,0.142561,1.000000,0.128947,0.155543,0.096469,0.238284,0.025164,0.026337,0.114083,0.092535,...,0.113155,0.087802,0.049454,0.107449,0.123916,0.101689,0.159258,0.008477,0.040535,0.056570
Adrenal_Gland_z,0.077067,0.128947,1.000000,0.126825,0.176840,0.084395,0.105605,0.094743,0.185800,0.120386,...,0.111314,0.098426,0.152262,0.191994,0.182184,0.170112,0.233740,0.077688,0.079411,0.037341
Artery_Aorta_z,0.216581,0.155543,0.126825,1.000000,0.151923,0.201480,0.111518,0.033746,0.190330,0.174112,...,0.057810,0.034831,0.241280,0.069360,0.147779,-0.010239,0.117677,0.060275,0.067869,0.052684
Artery_Coronary_z,0.090789,0.096469,0.176840,0.151923,1.000000,0.119469,0.152308,0.095471,0.164125,0.218160,...,0.293951,0.037284,0.207378,0.172723,0.135988,0.105529,0.169198,0.132848,0.145592,0.047626
Artery_Tibial_z,0.138014,0.238284,0.084395,0.201480,0.119469,1.000000,0.050010,0.042873,0.108598,-0.172340,...,0.167866,0.097994,0.025661,0.074162,0.153287,0.066544,0.126896,0.007951,0.010369,0.070341
Brain_Amygdala_z,0.027970,0.025164,0.105605,0.111518,0.152308,0.050010,1.000000,0.262328,0.173055,0.231170,...,0.040676,-0.001647,0.149913,0.105116,0.174091,0.002328,0.035640,0.255100,0.114306,0.003032
Brain_Anterior_cingulate_cortex_BA24_z,0.127328,0.026337,0.094743,0.033746,0.095471,0.042873,0.262328,1.000000,0.223412,0.236047,...,-0.002203,0.010010,0.080203,0.052420,0.057690,0.060204,0.011502,0.174675,0.286927,0.049808
Brain_Caudate_basal_ganglia_z,0.244253,0.114083,0.185800,0.190330,0.164125,0.108598,0.173055,0.223412,1.000000,0.246422,...,0.103076,0.029288,0.187260,0.134018,0.281965,0.117004,0.207459,0.197036,0.243360,0.133114
Brain_Cerebellar_Hemisphere_z,0.062549,0.092535,0.120386,0.174112,0.218160,-0.172340,0.231170,0.236047,0.246422,1.000000,...,0.039528,0.036676,0.263088,0.116246,0.057685,0.052726,0.054559,0.093672,0.061035,0.045663


In [23]:
eQTLs.corr().mean().mean()

0.1296866508212042

In [28]:
eQTLs['Mean_z']=eQTLs[eQTLs.columns.drop('Cells_EBV-transformed_lymphocytes_z')].mean(axis=1)

In [39]:
eQTLs.columns

Index([u'variant_id', u'gene_id', u'Adipose_Subcutaneous_z',
       u'Adipose_Visceral_Omentum_z', u'Adrenal_Gland_z', u'Artery_Aorta_z',
       u'Artery_Coronary_z', u'Artery_Tibial_z', u'Brain_Amygdala_z',
       u'Brain_Anterior_cingulate_cortex_BA24_z',
       u'Brain_Caudate_basal_ganglia_z', u'Brain_Cerebellar_Hemisphere_z',
       u'Brain_Cerebellum_z', u'Brain_Cortex_z', u'Brain_Frontal_Cortex_BA9_z',
       u'Brain_Hippocampus_z', u'Brain_Hypothalamus_z',
       u'Brain_Nucleus_accumbens_basal_ganglia_z',
       u'Brain_Putamen_basal_ganglia_z', u'Brain_Spinal_cord_cervical_c-1_z',
       u'Brain_Substantia_nigra_z', u'Breast_Mammary_Tissue_z',
       u'Cells_Cultured_fibroblasts_z', u'Cells_EBV-transformed_lymphocytes_z',
       u'Colon_Sigmoid_z', u'Colon_Transverse_z',
       u'Esophagus_Gastroesophageal_Junction_z', u'Esophagus_Mucosa_z',
       u'Esophagus_Muscularis_z', u'Heart_Atrial_Appendage_z',
       u'Heart_Left_Ventricle_z', u'Kidney_Cortex_z', u'Liver_z', u'Lung_

In [43]:
eQTLs=eQTLs.dropna(subset=['Mean_z']).fillna(0)

In [46]:
eQTLs

,variant_id,gene_id,Adipose_Subcutaneous_z,Adipose_Visceral_Omentum_z,Adrenal_Gland_z,Artery_Aorta_z,Artery_Coronary_z,Artery_Tibial_z,Brain_Amygdala_z,Brain_Anterior_cingulate_cortex_BA24_z,...,Skin_Sun_Exposed_Lower_leg_z,Small_Intestine_Terminal_Ileum_z,Spleen_z,Stomach_z,Testis_z,Thyroid_z,Uterus_z,Vagina_z,Whole_Blood_z,Mean
0,chr1_951408_G_A_b38,ENSG00000187961,-0.164405,-0.212897,2.677308,-0.283219,0,-0.192060,0,0,...,-0.200044,0,3.785873,0.000000,-0.263424,-0.185930,0.000000,0.000000,-0.196089,0.194649
1,chr1_965350_G_A_b38,ENSG00000187961,-0.167642,-0.212897,-0.274405,-0.283858,0,-0.192047,0,0,...,-0.200044,0,1.527755,0.000000,-0.263418,-0.186347,0.000000,0.000000,-0.196089,0.194928
2,chr1_966179_G_A_b38,ENSG00000187961,-0.167686,-0.212897,-0.296112,-0.283940,0,-0.192061,0,0,...,-0.200044,0,0.394595,2.472856,-0.263424,-0.186179,0.000000,0.000000,-0.196089,0.115658
3,chr1_1295039_T_C_b38,ENSG00000127054,-0.167627,-0.043291,0.000000,0.000000,0,2.091423,0,0,...,0.000000,0,0.000000,0.000000,-0.254634,-0.186448,0.000000,0.000000,0.000000,0.012531
4,chr1_1296939_G_A_b38,ENSG00000127054,-0.166803,-0.186047,2.517906,0.000000,0,2.369813,0,0,...,0.000000,0,0.000000,0.000000,-0.262037,-0.186451,0.000000,0.000000,3.617414,0.409923
5,chr1_1298561_T_C_b38,ENSG00000127054,-0.167627,-0.043291,0.000000,0.000000,0,2.091423,0,0,...,0.000000,0,0.000000,0.000000,-0.254634,-0.186448,0.000000,0.000000,0.000000,0.012531
6,chr1_1300330_T_A_b38,ENSG00000127054,-0.167752,-0.196805,0.473458,0.000000,0,0.121323,0,0,...,4.014885,0,0.000000,0.000000,-0.262382,-0.186451,0.000000,0.000000,4.336909,0.520579
7,chr1_1301656_T_C_b38,ENSG00000127054,-0.167752,-0.196805,0.473458,0.000000,0,0.121323,0,0,...,4.014885,0,0.000000,0.000000,-0.262382,-0.186451,0.000000,0.000000,4.336909,0.520579
8,chr1_1303959_T_G_b38,ENSG00000127054,0.167625,0.158010,-3.669250,-1.949034,0,-1.434795,0,0,...,0.000000,0,0.000000,0.000000,0.262037,0.186451,0.000000,0.000000,-2.994210,-0.557710
9,chr1_1306420_A_G_b38,ENSG00000127054,0.167746,0.098149,-3.371360,0.000000,0,-0.047399,0,0,...,0.000000,0,0.000000,0.000000,0.259755,0.186448,0.000000,0.000000,0.000000,-0.451598


In [29]:
eQTLs[['variant_id','gene_id','Mean_z']]

,variant_id,gene_id,Mean
0,chr1_951408_G_A_b38,ENSG00000187961,0.194649
1,chr1_965350_G_A_b38,ENSG00000187961,0.194928
2,chr1_966179_G_A_b38,ENSG00000187961,0.115658
3,chr1_1295039_T_C_b38,ENSG00000127054,0.012531
4,chr1_1296939_G_A_b38,ENSG00000127054,0.409923
5,chr1_1298561_T_C_b38,ENSG00000127054,0.012531
6,chr1_1300330_T_A_b38,ENSG00000127054,0.520579
7,chr1_1301656_T_C_b38,ENSG00000127054,0.520579
8,chr1_1303959_T_G_b38,ENSG00000127054,-0.557710
9,chr1_1306420_A_G_b38,ENSG00000127054,-0.451598


In [ ]:
eQTLs.to_csv(out_folder+'merged_eQTLs.csv')

# Gene intersection with Database

In [47]:
snp_db=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/GTEX/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.lookup_table.txt.gz',compression='gzip',sep='\t',usecols=['variant_id','rs_id_dbSNP151_GRCh38p7'])

In [48]:
snp_db=snp_db[snp_db['rs_id_dbSNP151_GRCh38p7']!= '.']

In [53]:
variant_list=pd.DataFrame(eQTLs['variant_id'].unique(),columns=['variant_id'])

In [52]:
# variant_list=pd.DataFrame(top_genes_eQTLs[['gene_id','variant_id']].merge(eQTLs[['gene_id','variant_id']])['variant_id'].unique(),columns=['variant_id'])

In [11]:
# variant_list=pd.DataFrame(top_genes_eQTLs['variant_id'].unique(),columns=['variant_id'])

In [54]:
GM12878_variants=variant_list.merge(snp_db,how='inner')

In [58]:
eQTLs_ready=GM12878_variants.merge(eQTLs)

In [64]:
eQTLs_ready['chr']=eQTLs_ready['variant_id'].str.split('_',expand=True)[0].str.lstrip('chr')

In [65]:
eQTLs_ready['pos']=eQTLs_ready['variant_id'].str.split('_',expand=True)[1].astype(int)

In [68]:
eQTLs_ready=eQTLs_ready[eQTLs_ready['chr']!='X']

In [69]:
eQTLs_ready['chr']=eQTLs_ready['chr'].astype(int)

/homes/jhidalgo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [71]:
eQTLs_ready.to_csv(out_folder+'eQTLs_z.csv.gz',compression='gzip',index=False)

In [76]:
eQTLs_ready

,variant_id,rs_id_dbSNP151_GRCh38p7,gene_id,Adipose_Subcutaneous_z,Adipose_Visceral_Omentum_z,Adrenal_Gland_z,Artery_Aorta_z,Artery_Coronary_z,Artery_Tibial_z,Brain_Amygdala_z,...,Spleen_z,Stomach_z,Testis_z,Thyroid_z,Uterus_z,Vagina_z,Whole_Blood_z,Mean,chr,pos
0,chr1_951408_G_A_b38,rs10465242,ENSG00000187961,-0.164405,-0.212897,2.677308,-0.283219,0,-0.192060,0,...,3.785873,0.000000,-0.263424,-0.185930,0.000000,0.000000,-0.196089,0.194649,1,951408
1,chr1_965350_G_A_b38,rs3935066,ENSG00000187961,-0.167642,-0.212897,-0.274405,-0.283858,0,-0.192047,0,...,1.527755,0.000000,-0.263418,-0.186347,0.000000,0.000000,-0.196089,0.194928,1,965350
2,chr1_966179_G_A_b38,rs13303160,ENSG00000187961,-0.167686,-0.212897,-0.296112,-0.283940,0,-0.192061,0,...,0.394595,2.472856,-0.263424,-0.186179,0.000000,0.000000,-0.196089,0.115658,1,966179
3,chr1_1295039_T_C_b38,rs1272959,ENSG00000127054,-0.167627,-0.043291,0.000000,0.000000,0,2.091423,0,...,0.000000,0.000000,-0.254634,-0.186448,0.000000,0.000000,0.000000,0.012531,1,1295039
4,chr1_1296939_G_A_b38,rs604618,ENSG00000127054,-0.166803,-0.186047,2.517906,0.000000,0,2.369813,0,...,0.000000,0.000000,-0.262037,-0.186451,0.000000,0.000000,3.617414,0.409923,1,1296939
5,chr1_1298561_T_C_b38,rs1739855,ENSG00000127054,-0.167627,-0.043291,0.000000,0.000000,0,2.091423,0,...,0.000000,0.000000,-0.254634,-0.186448,0.000000,0.000000,0.000000,0.012531,1,1298561
6,chr1_1300330_T_A_b38,rs640712,ENSG00000127054,-0.167752,-0.196805,0.473458,0.000000,0,0.121323,0,...,0.000000,0.000000,-0.262382,-0.186451,0.000000,0.000000,4.336909,0.520579,1,1300330
7,chr1_1301656_T_C_b38,rs2477782,ENSG00000127054,-0.167752,-0.196805,0.473458,0.000000,0,0.121323,0,...,0.000000,0.000000,-0.262382,-0.186451,0.000000,0.000000,4.336909,0.520579,1,1301656
8,chr1_1303959_T_G_b38,rs11260584,ENSG00000127054,0.167625,0.158010,-3.669250,-1.949034,0,-1.434795,0,...,0.000000,0.000000,0.262037,0.186451,0.000000,0.000000,-2.994210,-0.557710,1,1303959
9,chr1_1306420_A_G_b38,rs3737720,ENSG00000127054,0.167746,0.098149,-3.371360,0.000000,0,-0.047399,0,...,0.000000,0.000000,0.259755,0.186448,0.000000,0.000000,0.000000,-0.451598,1,1306420


# LD calculation and file prep

In [3]:
eQTLs_ready=pd.read_csv(out_folder+'eQTLs_z.csv.gz',compression='gzip')

In [24]:
# eQTLs_ready=eQTLs_ready.rename(columns={'Mean':'Mean_z'})

In [8]:
for i in tissues+['Mean']:
    folder='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/Z/'+i+'/'
    if not os.path.exists(folder):os.makedirs(folder)

In [6]:
for i in ['LD_R','LD_R2']:
    folder='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/'+i+'/'
    if not os.path.exists(folder):os.makedirs(folder)

# Postgap

In [9]:
import psutil

In [10]:
def close_files():
    closing_files = []
    for fd in psutil.Process().open_files():
        if 'inteql_GTEX_v8' in fd.path:
            closing_files.append(fd)
    for fd in closing_files:
        open(fd.fd).close()

In [11]:
# top_genes_eQTLs = pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/eQTL_before_finemapping.csv')

In [12]:
postgap.Globals.DATABASES_DIR = '/nfs/research1/zerbino/jhidalgo/databases'

In [13]:
postgap.Globals.SPECIES = 'human'

In [14]:
server = "http://rest.ensembl.org"
postgap.Globals.SPECIES = 'human'
rs_name = 'rs_id_dbSNP151_GRCh38p7'

In [ ]:
startingtime=time.time()
failed=[]
length=len(eQTLs_ready['gene_id'].unique())
counter=1
for i in eQTLs_ready['gene_id'].unique():
    filename=str(i)+'_LD.csv'
    if os.path.exists('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/LD_R2/'+filename): 
        print(str(i)+' File found, skipping.')
        counter=counter+1
        continue
    startlooptime=time.time()
    subdata=eQTLs_ready[eQTLs_ready['gene_id']==i]
    snps=postgap.Ensembl_lookup.get_snp_locations(subdata['rs_id_dbSNP151_GRCh38p7'].unique())
    if snps == []:
        print(str(i)+' block could not be processed.')
        failed=failed+[i]
        counter=counter+1
        continue
    rsIDs, matrix = postgap.LD.get_pairwise_ld(snps, 'EUR')    
    if not matrix.size: 
        print(str(i)+' block could not be processed.')
        failed=failed+[i]
        counter=counter+1
        continue
    for tissue in tissues+['Mean']:
        eQTL_output=subdata[subdata['rs_id_dbSNP151_GRCh38p7'].isin(rsIDs)][['rs_id_dbSNP151_GRCh38p7',tissue+'_z']]
        eQTL_output.to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/Z/'+tissue+'/'+i+'_z.csv',index=False,sep='\t')
    pd.DataFrame(matrix*matrix,columns=rsIDs,index=rsIDs).to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/LD_R2/'+filename,index=True,header=False,sep='\t')
    pd.DataFrame(matrix,columns=rsIDs,index=rsIDs).to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/Finemapping/LD_R/'+filename,index=True,header=False,sep='\t')
    del [eQTL_output,subdata,snps,rsIDs, matrix,filename]
    endlooptime=time.time()
    deltatimeloop=endlooptime-startlooptime
    deltatime=endlooptime-startingtime
    print(str(counter)+'/'+str(length)+'. '+str("%.2f" % deltatime) + ' seconds after start, computing for '+str("%.2f" % deltatimeloop)+' seconds, block '+str(i)+' has finished.')
    counter=counter+1
    close_files()

ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!


ENSG00000187961 block could not be processed.
ENSG00000127054 File found, skipping.
ENSG00000242485 File found, skipping.
ENSG00000160072 File found, skipping.
ENSG00000197530 block could not be processed.
ENSG00000189339 File found, skipping.
ENSG00000008128 File found, skipping.
ENSG00000157873 block could not be processed.
ENSG00000238164 File found, skipping.
ENSG00000116198 block could not be processed.
ENSG00000116237 block could not be processed.


ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!


ENSG00000049239 block could not be processed.
ENSG00000127463 block could not be processed.
ENSG00000053372 block could not be processed.
ENSG00000053371 block could not be processed.
ENSG00000090432 block could not be processed.


ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!
ERROR:root:LD matrix is identity matrix!


...

ENSG00000089916 block could not be processed.


ERROR:root:LD matrix is identity matrix!


ENSG00000198668 block could not be processed.


ERROR:root:LD matrix is identity matrix!


ENSG00000165929 block could not be processed.


In [28]:
counter

1608

# Postgap OLD

In [82]:
import psutil

In [83]:
def close_files():
    closing_files = []
    for fd in psutil.Process().open_files():
        if 'inteql_GTEX_v8' in fd.path:
            closing_files.append(fd)
    for fd in closing_files:
        open(fd.fd).close()

In [29]:
top_genes_eQTLs = pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/eQTL_before_finemapping.csv')

In [30]:
postgap.Globals.DATABASES_DIR = '/nfs/research1/zerbino/jhidalgo/databases'

In [31]:
postgap.Globals.SPECIES = 'human'

In [32]:
server = "http://rest.ensembl.org"
postgap.Globals.SPECIES = 'human'
rs_name = 'rs_id_dbSNP151_GRCh38p7'

In [41]:
### NEW GENE SPLIT
startingtime=time.time()
failed=[]
length=len(top_genes_eQTLs['gene_id'].unique())
counter=1
for i in top_genes_eQTLs['gene_id'].unique():
    filename=str(i)+'_LD.csv'
    if os.path.exists('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/gene_split_finemapping/LD_R2/'+filename): 
        print(str(i)+' File found, skipping.')
        counter=counter+1
        continue
    startlooptime=time.time()
    subdata=top_genes_eQTLs[top_genes_eQTLs['gene_id']==i]
    snps=postgap.Ensembl_lookup.get_snp_locations(subdata['rs_id_dbSNP151_GRCh38p7'].unique())
    if snps == []:
        print(str(i)+' block could not be processed.')
        failed=failed+[i]
        counter=counter+1
        continue
    rsIDs, matrix = postgap.LD.get_pairwise_ld(snps, 'EUR')    
    if not matrix.size: 
        print(str(i)+' block could not be processed.')
        failed=failed+[i]
        counter=counter+1
        continue
    eQTL_output=subdata[subdata['rs_id_dbSNP151_GRCh38p7'].isin(rsIDs)][['rs_id_dbSNP151_GRCh38p7','slope','pval_nominal']]
    eQTL_output['slope'][eQTL_output['slope'] > 0] = 1
    eQTL_output['slope'][eQTL_output['slope'] < 0] = -1
    eQTL_output['pval_nominal']=stats.zscore(eQTL_output['pval_nominal'])*eQTL_output['slope']
    eQTL_output.iloc[:,[0,2]].to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/gene_split_finemapping/eQTL_z/'+str(i)+'.csv',index=False,sep='\t')
    pd.DataFrame(matrix*matrix,columns=rsIDs,index=rsIDs).to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/gene_split_finemapping/LD_R2/'+filename,index=True,header=False,sep='\t')
    pd.DataFrame(matrix,columns=rsIDs,index=rsIDs).to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/gene_split_finemapping/LD_R/'+filename,index=True,header=False,sep='\t')
    del [eQTL_output,subdata,snps,rsIDs, matrix,filename]
    endlooptime=time.time()
    deltatimeloop=endlooptime-startlooptime
    deltatime=endlooptime-startingtime
    print(str(counter)+'/'+str(length)+'. '+str("%.2f" % deltatime) + ' seconds after start, computing for '+str("%.2f" % deltatimeloop)+' seconds, block '+str(i)+' has finished.')
    counter=counter+1
    close_files()

ERROR:root:LD matrix is identity matrix!


ENSG00000225972 block could not be processed.


ERROR:root:LD matrix is identity matrix!


ENSG00000237973 block could not be processed.


ERROR:root:LD matrix is identity matrix!


...

ENSG00000100288 block could not be processed.


ERROR:root:LD matrix is identity matrix!


ENSG00000100299 block could not be processed.


ERROR:root:LD matrix is identity matrix!


ENSG00000079974 block could not be processed.


In [42]:
counter

3445